Calcula la suma y multiplicación de las velocidades para cada muestra en diferentes cortes de eventos

In [1]:
import pandas as pd
from pathlib import Path
import os

def calculate(filename):
    directory = os.path.join(Path().absolute(), "speed-prediction-logs")
    path = os.path.join(directory, "TimeWindow")
    path = os.path.join(path, filename + ".csv")
    df = pd.read_csv(path)

    df = df.fillna(0)
    
    df["sum" + filename] = df["avgSpeedPerTest"] + df["avgIntratecla"] + df["avgScrollSpeed"]
    df["mul" + filename] = df["avgSpeedPerTest"] * df["avgIntratecla"] * df["avgScrollSpeed"]
    df["mouseSpeed" + filename] = df["avgSpeedPerTest"]
    df["keySpeed" + filename] = df["avgIntratecla"]
    df["scrollSpeed" + filename] = df["avgScrollSpeed"]

    column_names = df.columns.values.tolist()
    column_names.remove('name')
    column_names.remove("sum" + filename)
    column_names.remove("mul" + filename)
    column_names.remove("mouseSpeed" + filename)
    column_names.remove("keySpeed" + filename)
    column_names.remove("scrollSpeed" + filename)
    for name in column_names:
        df.pop(name)
    
    return df

directory = os.path.join(Path().absolute(), "speed-prediction-logs")
path = os.path.join(directory, "TimeWindow")
path = os.path.join(path, "all.csv")
df = pd.read_csv(path)

logs = range(5000,300000,5000)
for name in logs:
    df = df.merge(right=calculate(str(name)), on='name', how='left')
df = df.merge(right=calculate("all"), on='name', how='left')

speed_df = df.copy(deep=True)

for name in logs:
    name = str(name)
    df["sum"+name] = df["sum"+name]  / df["sum120000"]
    df["mul"+name] = df["mul"+name]  / df["mul120000"]

speed_df_norm = df
#speed_df_norm.to_csv('speed_prediction_normalized.csv')

In [ ]:
speed_df

Observar los números calculados por suma

In [ ]:
def remove_columns_that_starts_with(df, prefix):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if name.startswith(prefix):
            df.pop(name)
    return df

speed_df_only_sum = speed_df.copy(deep=True)
speed_df_only_sum = remove_columns_that_starts_with(speed_df_only_sum, "mul")
speed_df_only_sum

In [ ]:
speed_df_only_sum.to_csv('speed_df_only_sum.csv')
speed_df_only_sum.describe().to_csv('speed_df_only_sum_describe.csv')
speed_df_only_sum.describe()

In [ ]:
speed_df_only_mouse = speed_df.copy(deep=True)
speed_df_only_mouse = remove_columns_that_starts_with(speed_df_only_mouse, "mouseSpeed")
speed_df_only_mouse

Ver cuanto se acercan los valores de la suma al final

In [ ]:
df_test = speed_df_only_sum.copy(deep=True)
column_names = df_test.columns.values.tolist()
for name in column_names:
    if "all" in name:
        continue
    if name.startswith("sum"):
        amount = name.replace("sum","")
        amount = int(amount)
        df_test.loc[amount > df["timePerTest"], name] = None

        df_test[name] = df_test[name] / df_test["sumall"]

def remove_columns_that_dont_have(df, word):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if word not in name:
            df.pop(name)
    return df

df_test = remove_columns_that_dont_have(df_test, "sum")

df_test.to_csv('df_test_normalized.csv')
df_test

In [ ]:
df_test.describe().to_csv('df_test_normalized_describe.csv')
df_test.describe()

In [2]:
def remove_columns_that_dont_have(df, word):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if name == "name":
            continue
        if word not in name:
            df.pop(name)
    return df

def normalize_df_and_save(df, word):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if "all" in name:
            continue
        if name.startswith(word):
            amount = name.replace(word,"")
            amount = int(amount)
            df.loc[amount > df["timePerTest"], name] = None

            df[name] = df[name] / df[word + "all"]

    df = remove_columns_that_dont_have(df, word)
    df.to_csv(word + '.csv')
    df.describe().to_csv(word + 'Describe.csv')
    return df

df_mouse = normalize_df_and_save(speed_df.copy(deep=True), "mouseSpeed")
df_scroll = normalize_df_and_save(speed_df.copy(deep=True), "scrollSpeed")
df_key = normalize_df_and_save(speed_df.copy(deep=True), "keySpeed")

In [3]:
df_combinated = df_mouse.copy(deep=True)
df_combinated = df_combinated.merge(right=df_scroll, on='name', how='left')
df_combinated = df_combinated.merge(right=df_key, on='name', how='left')

for i in range(5000,300000,5000):
    df_combinated["sum" + str(i)] = df_combinated["keySpeed" + str(i)] + df_combinated["mouseSpeed" + str(i)] + df_combinated["scrollSpeed" + str(i)]

df_combinated = remove_columns_that_dont_have(df_combinated, "sum")
df_combinated.to_csv('df_combinated.csv')
df_combinated.describe().to_csv('df_combinatedDescribe.csv')